In [8]:
import math as m
import numpy as np

In [9]:
from typing import List, Tuple, Dict
import numpy as np

TSwitch1 = 1.03
TSwitch2 = 4.502
t_min = 0
t_max = 5

In [48]:
class OptimalControl:
    def __init__(
        self, 
        u1: float,
        u2: float, 
        frag: int,
        t_min: float,
        t_max: float,
        n_task: int,
        ):
        self.u1 = u1
        self.u2 = u2
        self.frag = frag
        self.t_min = t_min
        self.t_max = t_max
        self.n_task = n_task #number of task
    
    
    def CalcControl(self, t_switch):
        res1, res2 = self.RungeK(t_switch[0], t_switch[1])
        if self.n_task == 0:
            return [res1, res2], res1**2 + res2**2
        else:
            return [res1, res2], -res1 + 0.5 * res2
    
    
    def task1(self, x1, x2, u_switch):
        return x2, u_switch - m.sin(x1)
    
    def RungeK(self, t1_switch, t2_switch):
        TSwitch1 = t1_switch
        TSwitch2 = t2_switch
        x1 = 5
        x2 = 0
        h = (self.t_max - self.t_min) / self.frag
        u_switch = self.u1
        for t in np.arange(self.t_min, self.t_max, h):
            if t >= TSwitch1:
                u_switch = self.u2
                if t >= TSwitch2:
                    u_switch = self.u1

            k1, l1 = self.task1(x1, x2, u_switch)
            k2, l2 = self.task1(x1 + h / 2.0, x2 + l1 * h /2.0, u_switch)
            k3, l3 = self.task1(x1 + h / 2.0, x2 + l2 * h /2.0, u_switch)
            k4, l4 = self.task1(x1 + h, x2 + l3 * h, u_switch)
        
            x1 = x1 + (k1 + 2 * k2 + 2 * k3 + k4) * h / 6.0
            x2 = x2 + (l1 + 2 * l2 + 2 * l3 + l4) * h / 6.0
        return x1, x2

In [118]:
class Dragonfly:
    def __init__(
        self, 
        x: List[float], 
        t: List[float],
        f: float,
             ):
        self.x = x
        self.t = t
        self.f = f
        
class DragonflySwarm:
    def __init__(
        self,
        swarm_size: int,
        dim: int,
        max_iter: int,
        sep: float,
        alignmnt: float,
        cohesn: float,
        tendnsy: float,
        evatn: float,
        weight: float, 
        alpha: float,
        lmbd: float,
        radius: float,
        area: List[List[float]],
                 ):
        self.swarm_size = swarm_size
        self.dim = dim
        self.max_iter = max_iter
        self.current_iter = 0
        self.sep = sep
        self.alignmt = alignmnt
        self.cohesn = cohesn
        self.tendnsy = tendnsy
        self.evatn = evatn
        self.weight = weight
        self.alpha = alpha
        self.lmbd = lmbd
        self.radius = radius
        self.area = area
        self.population = []
        self.sep_list = []
        self.alig_list = []
        self.coh_list = []
        self.food_attract = []
        self.enemy_avoid = []
        self.velocity_list =[]
        self.euclid_list = []
        self.best_list = []
        self.worst_list = []
        self.pathx_list = []
    
    def form_first_pop(self, control):
        for i in range(self.swarm_size):
            switch_moments = np.zeros(self.dim)
            switch_moments[0] =  np.random.rand() * (self.area[0][1] - self.area[0][0]) + self.area[0][0]
            for j in range(1, self.dim):
                switch_moments[j] = np.random.rand() * (self.area[j][1] - switch_moments[j - 1]) + switch_moments[j - 1]
            
            result = control.CalcControl(switch_moments)
            coordinate = np.zeros(self.dim)
            pathx = np.zeros(self.dim)
            for j in range(self.dim):
                coordinate[j] = result[0][j]
                pathx[j] = result[0][j]
            member = Dragonfly(coordinate, switch_moments, result[1])
            self.population.append(member)
            self.pathx_list.append(pathx)

    def generate_dragonfly(self, i, j):
        self.population[i][j] = np.random.rand() * (self.area[j][1] - self.population[i].t[j -1]) + self.population[i].t[j - 1]
        
    def generate_step_coord(self):
        for i in range(self.swarm_size):
            rj = np.random.uniform(-1, 1)
            velocty = [(self.area[j][1] - self.area[j][0]) * rj / 1000
                for j in range(self.dim)]
            self.velocity_list.append(velocty)
            
    def move_swarm(self, control):
        right_area = False # same as "flag" in a previous C# code
        population_copy = self.population.copy() 
        radius_area = self.radius * self.current_iter / self.max_iter
        for i in range(self.swarm_size):
            for j in range(self.swarm_size):
                if i != j:
                    dist = np.linalg.norm(np.array(population_copy[i].t) -
                                          np.array(population_copy[j].t))
                    if dist <= radius_area:
                        euclid_list.append([j, dist])
            if len(self.euclid_list) > 1:
                #print('NOT a LEVI')
                self.sep_list = self.separation(i)
                self.alig_list = self.alignment(i)
                self.coh_list = self.cohesion(i)
                self.food_attract = self.tendensy(i)
                self.enemy_avoid = self.evation(i)
                self.velocity_list.append(velocity(i))
                self.population.x = self.find_new_position(i, control) #wrong?????
                self.check_area(i)
            else:
                half_set = int(self.dim / 2)
                r_parameter = []
                theta = []
                l_parameter = []
                xi_parameter = []
                
                
                for ii in range(self.dim):
                    #print('ii:', ii, 'i:', i)
                    iterator_0 = 0
                    while not right_area:
                        r_parameter.append(np.random.rand() * (self.area[ii][1] - self.area[ii][0]) + self.area[ii][0])
                        theta.append(r_parameter[len(r_parameter) - 1] * 2 * np.pi)
                        l_parameter.append(r_parameter[len(r_parameter) - 1] ** self.lmbd)

                        if ii < half_set:
                            xi_parameter.append(l_parameter[len(l_parameter) - 1] * np.sin(theta[len(theta) - 1]))
                        else:
                            xi_parameter.append(l_parameter[len(l_parameter) - 1] * np.cos(theta[len(theta) - 1]))
                        

                        check_obl = np.array(self.population[i].t[ii]) + \
                        (self.alpha / (self.current_iter + 1)) * xi_parameter[len(xi_parameter) - 1]
                        #print(check_obl)
                        if (check_obl < self.area[ii][0]) or (check_obl > self.area[ii][1]):
                            right_area = False
                        else:
                            right_area = True
                        iterator_0 += 1
                    #print(iterator_0)
                    right_area = False
                    self.population[i].t[ii] = (self.population[i].t[ii]) + \
                        (self.alpha / (self.current_iter + 1)) * xi_parameter[len(xi_parameter) - 1]
            switch_moments = np.zeros(self.dim)
            for jj in range(self.dim):
                switch_moments[jj] = self.population[i].t[jj]
            for jj in range(self.dim - 1):
                if switch_moments[jj + 1] < switch_moments[jj]:
                    switch_moments[jj + 1] = np.random.rand() * (self.area[jj + 1][1] - switch_moments[jj]) + switch_moments[jj]
            result = control.CalcControl(switch_moments)
            coordinate = np.zeros(self.dim)
            pathx = np.zeros(self.dim)
            for jj in range(self.dim):
                coordinate[jj] = result[0][jj]
                pathx[jj] = result[0][jj]
            member = Dragonfly(coordinate, switch_moments, result[1])
            self.population[i] = member
            self.pathx_list[i] = pathx  
            self.euclid_list.clear()
            
    def separation(self, it):
        tmp = np.zeros(self.dim)
        #print(len(self.euclid_list))
        for indx, dist in self.euclid_list:
            tmp += (np.array(self.population[it].t) - np.array(self.population[indx].t))
            #print(tmp)
        for i in range(self.dim):
            area_tmp = self.area[i][1] - self.area[i][0]
            while (np.abs(tmp[i]) >= area_tmp):
                tmp[i] /= 10
            tmp[i] = -tmp[i]
        return tmp.tolist()
    
    def alignment(self, it):
        tmp = np.zeros(self.dim)
        for indx, dist in self.euclid_list:
            tmp += np.array(self.velocity_list[indx].x)
        tmp = tmp / len(self.euclid_list)
        for i in range(self.dim):
            area_tmp = self.area[i][1] - self.area[i][0]
            while (np.abs(tmp[i]) >= area_tmp):
                tmp[i] = tmp[i] / 10
        return tmp.tolist()
    
    def cohesion(self, it):
        tmp = np.zeros(self.dim)
        for indx, dist in self.euclid_list:
            tmp += np.array(self.population[indx].t)
        tmp = tmp / len(self.euclid_list) - np.array(self.population[i].t)
        for i in range(self.dim):
            area_tmp = self.area[i][1] - self.area[i][0]
            while (np.abs(tmp[i]) >= area_tmp):
                tmp[i] = tmp[i] / 10
        return tmp.tolist()
    
    def tendensy(self, it):
        best_p = min(self.population, key=lambda t: t[1])
        tmp = np.array(best_p.t) - np.array(self.population[it].t)
        for i in range(self.dim):
            area_tmp = self.area[i][1] - self.area[i][0]
            while (np.abs(tmp[i]) >= area_tmp):
                tmp[i] = tmp[i] / 10
        return tmp.tolist()
    
    def evation(self, it):
        worst_p = max(self.population, key=lambda t: t[1])
        tmp = np.array(worst_p.t) + np.array(self.population[it].t)
        for i in range(self.dim):
            area_tmp = self.area[i][1] - self.area[i][0]
            while (np.abs(tmp[i]) >= area_tmp):
                tmp[i] = tmp[i] / 10
        return tmp.tolist()
    
    def velocity(self, it):
        self.velocity_list[it] = (self.sep * np.array(self.sep_list[it]) + self.alignmnt * np.array(self.alig_list[it]) + self.cohesn * np.array(self.coh_list[it]) +\
        self.tendnsy * np.array(self.food_attract[it]) + self.evatn * np.array(self.enemy_avoid[it]) + self.weight * np.array(self.velocity_list[it])).tolist()
    
    def find_new_position(self, it, control):
        self.population[it].t = (np.array(self.population[it].t) + np.array(self.velocity_list[it])).tolist()
        for indx in range(self.dim - 1):
            if self.population[it].t[indx + 1] < self.population[it].t[indx]:
                self.population[it].t[indx + 1] = np.random.rand() * (self.area[indx + 1][1] - self.population[it].t[indx]) + self.population[it].t[indx]
        result = control.CalcControl(self.population[it].t)
        coordinate = np.zeros(self.dim)
        switch_moments = np.zeros(self.dim)
        for j in range(self.dim):
            coordinate[j] = result[0][j]
            switch_moments[j] = self.population[it].t[j]
        member = Dragonfly(coordinate, switch_moment, result[1])    
        self.population[it] = member
            
    def check_area(self, i):
        pass
    
    def find_best(self):
        return max(self.population, key=lambda dragonfly: dragonfly.f)
    
    def find_worst(self):
        return min(self.population, key=lambda dragonfly: dragonfly.f)
    
    def Work(self, control):
        for i in range(algorithm.max_iter):
            algorithm.move_swarm(control)
            best_temp = algorithm.find_best()
            tmp = np.zeros(self.dim)
            tmp_t = np.zeros(self.dim)
            for j in range(self.dim):
                tmp[j] = best_temp.x[j]
                tmp_t[j] = best_temp.t[j]
            member_temp = Dragonfly(tmp, tmp_t, best_temp.f)
            self.best_list.append(member_temp)
            tmp = np.zeros(self.dim)
            tmp_t = np.zeros(self.dim)
            worst_temp = algorithm.find_worst()
            for j in range(self.dim):
                tmp[j] = worst_temp.x[j]
                tmp_t[j] = worst_temp.t[j]
            member_temp = Dragonfly(tmp, tmp_t, worst_temp.f)
            self.worst_list.append(member_temp)
            
        self.best_list.sort(reverse=True, key=lambda dragonfly: dragonfly.f)
        self.worst_list.sort(key=lambda dragonfly: dragonfly.f)
        #print(self.best_list[0], self.worst_list[0])
                     

In [119]:
algorithm = DragonflySwarm(50, 2, 70, 0.9, 0.67, 0.4, 0.51, 0.1, 0.1, 1, 1, 0.8, [[-5, 5], [-5, 5]])

In [120]:
control = OptimalControl(1, -1, 1000, 5, -5, 0)

In [107]:
algorithm.form_first_pop(control)

In [108]:
#for i in range(len(algorithm.population)):
   # print(algorithm.population[i].t, algorithm.population[i].x, algorithm.population[i].f)

In [109]:
algorithm.generate_step_coord()

In [110]:
best = algorithm.find_best()
worst = algorithm.find_worst()

In [111]:
print(best.t, best.x, best.f)
print(worst.t, worst.x, worst.f)

[4.52057914 4.52762914] [ 58.31641139 -10.28826792] 3506.6522937064856
[2.01139813 4.8561933 ] [-0.40282578  3.49209108] 12.35696869063834


In [89]:
member_temp = Dragonfly(best.x, best.t, best.f)
algorithm.best_list.append(member_temp)
member_temp = Dragonfly(worst.x, worst.t, worst.f)
algorithm.worst_list.append(member_temp)

In [102]:
algorithm.move_swarm(control)

In [1]:
#for i in range(len(algorithm.population)):
 #   print(algorithm.population[i].t, algorithm.population[i].x, algorithm.population[i].f)

In [112]:
best = algorithm.find_best()
worst = algorithm.find_worst()

In [113]:
print(best.t, best.x, best.f)
print(worst.t, worst.x, worst.f)

[4.52057914 4.52762914] [ 58.31641139 -10.28826792] 3506.6522937064856
[2.01139813 4.8561933 ] [-0.40282578  3.49209108] 12.35696869063834


In [100]:
member_temp = Dragonfly(best.x, best.t, best.f)
algorithm.best_list.append(member_temp)
member_temp = Dragonfly(worst.x, worst.t, worst.f)
algorithm.worst_list.append(member_temp)

In [101]:
print(algorithm.worst_list[1].x, algorithm.worst_list[1].t, algorithm.worst_list[1].f)

[2.92043753 1.30870239] [2.72623122 4.96845037] 10.241657325481484


In [114]:
for member in algorithm.worst_list:
    print(member.t, member.x, member.f)
#NEED TO BE FIXED!!!

[2.01139813 4.8561933 ] [-0.40282578  3.49209108] 12.35696869063834


In [147]:
algorithm = DragonflySwarm(40, 2, 70, 0.5, 0.7, 0.4, 0.51, 0.1, 0.1, 1, 1, 0.8, [[-5, 5], [-5, 5]])
control = OptimalControl(1, -1, 100, 5, -5, 0)

In [148]:
algorithm.form_first_pop(control)
algorithm.generate_step_coord()

best = algorithm.find_best()
worst = algorithm.find_worst()

member_temp = Dragonfly(best.x, best.t, best.f)
algorithm.best_list.append(member_temp)
member_temp = Dragonfly(worst.x, worst.t, worst.f)
algorithm.worst_list.append(member_temp)

algorithm.Work(control)


print(algorithm.worst_list[0].x, algorithm.worst_list[0].t, algorithm.worst_list[0].f)
print('Отклонение от точного решениия:', 11.90817 - algorithm.worst_list[0].f)

[2.92043753 1.30870239] [4.22701978 3.72436273] 10.241657325481484
Отклонение от точного решениия: 1.6665126745185166


In [2]:
#for member in algorithm.worst_list:
 #   print(member.x, member.t, member.f)

In [ ]:
#for i in range(len(algorithm.population)):
 #   print(algorithm.population[i].x, algorithm.population[i].f)

#for i in (algorithm.velocity_list):
   # print(i)

#for i in range(len(algorithm.population)):
    #print(algorithm.population[i].x, algorithm.population[i].f)
    
#for member in algorithm.best_list:
   # print(member.x, member.f)